In [ ]:
# https://github.com/facebookresearch/mae/blob/main/models_mae.py
# https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py
# https://arxiv.org/abs/2010.11929
# https://arxiv.org/abs/2106.10270
# https://arxiv.org/abs/2212.08013
# https://github.com/lucidrains/vit-pytorch

In [ ]:
# http://127.0.0.1:8083/notebooks/watermarking/notebooks/v-6/Hidden_discriminator_01_18_modified_output_2_28_restarting.ipynb

In [ ]:
# !pip install einops
# !pip install transformers
# https://github.com/FrancescoSaverioZuppichini/ViT#position-embedding
# !pip install pytorch_ssim

In [1]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab, vocab
from torchtext.utils import download_from_url, extract_archive
import io

import random
from typing import Tuple

import torch.nn as nn
from torch.nn import Transformer
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

import pdb

import torch
import torch.nn as nn
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset

from torchvision import datasets as dv
from torchtext import datasets as dt
from torchtext.vocab import vocab, GloVe
from torchtext.data.utils import get_tokenizer

from torchtext.data.metrics import bleu_score

import numpy as np

from torchinfo import summary
import string
import os
import glob
import random
from matplotlib import pyplot as plt
from PIL import Image

from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torchinfo import summary
import re
from torchmetrics import StructuralSimilarityIndexMeasure as SSIM
import random
from skimage.util import random_noise

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# from torchsummary import summary

import torchvision.transforms as transforms
import math
from transformers import ViTFeatureExtractor, ViTModel
import pytorch_ssim

In [2]:
device = DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List


url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

# Place-holders
SRC_LANGUAGE = TGT_LANGUAGE = 'eng'
token_transform = {}
vocab_transform = {}

In [5]:
def preprocess(text_):
    text = text_.lower()
    text = re.sub(r'\+d', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.strip()
    text = text.split(" ")[:14]
#     pdb.set_trace()
    # if len(text) != 14:
    #         text.extend((14-len(text))*["[PAD]"])
    text = ' '.join(i for i in text)

    return text


def prepare_data(file_path, save_file):
    with open(file_path, 'rb') as f_in:
        content = f_in.read().decode('utf-8').split('\n')

        preprocessed_content = [preprocess(i) for i in content]
        preprocess_content_ = '\n'.join(i for i in preprocessed_content)
#         print(preprocessed_content, preprocess_content_)
        with open(save_file, 'w', encoding='utf-8') as f:
            f.write(preprocess_content_)
            f.close()
        f_in.close()

prepare_data(train_filepaths[1], 'train.txt')
prepare_data(val_filepaths[1], 'val.txt')

In [6]:
token_transform[SRC_LANGUAGE] = token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# helper function to yield list of tokens
def yield_tokens(file_path):
    with io.open(file_path, encoding='utf-8') as f:
        for line in f:
            yield token_transform[SRC_LANGUAGE](line.strip())


# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
file_path = './train.txt'
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(file_path),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

In [7]:
class TextDataset(Dataset):

    def __init__(self, inp_file):
        print("Loading plain data ...")
        self.inp = []

        # Read the EN lines
        num_inp_lines = 0
        with open(inp_file, "r") as ef:
            for line in ef:
                self.inp.append(line)
                num_inp_lines += 1

    def __getitem__(self, offset):
        en = self.inp[offset]
        return en

    def __len__(self):
        return len(self.inp)

In [8]:
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)
    
# Encoder
class Encoder(nn.Module):
    def __init__(self,
                 emb_size: int,
                 tgt_vocab_size: int,
                 transformer, 
                 positional_encoding,
                 dropout: float = 0.1):
        super(Encoder, self).__init__()
        self.transformer = transformer
#         self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
#         self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = positional_encoding
        self.vocab_size = tgt_vocab_size

    def forward(self,
                src: Tensor,
                src_mask: Tensor,
                src_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        memory = self.transformer.encoder(src_emb, src_mask, src_padding_mask)
        return memory
    
    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

# Encoder
class Decoder(nn.Module):
    def __init__(self,
                 emb_size: int,
                 tgt_vocab_size: int,
                 transformer, 
                 positional_encoding,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Decoder, self).__init__()
        self.transformer = transformer
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = positional_encoding
        self.vocab_size = tgt_vocab_size

    def forward(self,
                trg: Tensor,
                memory: Tensor,
                tgt_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer.decoder(tgt_emb, memory, tgt_mask=tgt_mask, tgt_key_padding_mask=tgt_padding_mask, memory_key_padding_mask=memory_key_padding_mask)
        return self.generator(outs)
    
    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask



In [9]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    if len(token_ids) < 14:
            token_ids.extend((14-len(token_ids))*[PAD_IDX])
    else:
        token_ids = token_ids[:14]
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
#         tgt_batch.append(text_transform[TGT_LANGUAGE](src_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
#     tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch

In [10]:
def remove_image(path_):
    remove_list = []
    for i in glob.glob(path_+'*'):
        img = np.array(Image.open(os.path.join(path_,i)))
        if img.shape[-1] != 3:
            remove_list.append(os.path.join(path_,i))
    return remove_list

In [11]:
class WatermarkDataset(Dataset):
  def __init__(self, cover_img_dir, remove_list, transform_c=None, data_size=None):
#     self.tokens = []
    self.cover_img_dir = cover_img_dir
    self.ls = []
    self.transform_c = transform_c
#     for sentence in watermark_text:
#         self.tokens.append(torch.cat([torch.tensor([BOS_IDX]), sentence, torch.tensor([EOS_IDX])], dim=0))
    
#     self.tokens = pad_sequence(self.tokens, padding_value=1)
    
#     self.input_ids = torch.tensor(self.tokens, dtype=torch.long)
    for i in os.listdir(self.cover_img_dir):
            if i.split('.')[-1] == 'JPEG' or 'jpg':
                self.ls.append(os.path.join(self.cover_img_dir,i))
    for i in glob.glob(self.cover_img_dir+'*/*'):
          self.ls.append(i)
                
    self.remove_list = remove_list
    self.ls = list(set(self.ls) - set(self.remove_list))
    
    random.shuffle(self.ls)
    
    self.cover_image_list = self.ls[:data_size]
#     self.input_ids = self.tokens[:data_size]

  def __len__(self):
    return len(self.cover_image_list)
  
  def __getitem__(self, index):
#     input_ids = self.input_ids[:, index]
    cover_img_pt = self.cover_image_list[index]

    cover_img = Image.open(cover_img_pt)
    cover_img = T.ToTensor()(cover_img)

    if self.transform_c:
        cover_img = self.transform_c(cover_img)
        cover_img = T.ToTensor()(cover_img)
    return cover_img#, input_ids

In [12]:
train_dir = '/home/bishwa/coco_dataset/train/'
val_dir = '/home/bishwa/coco_dataset/val/'
# test_dir = '/home/bishwa/coco_dataset/test/'
train_remove = remove_image(train_dir)
val_remove = remove_image(val_dir)
# test_remove = remove_image(test_dir)

train_set = WatermarkDataset(train_dir, train_remove,
                                        transform_c=T.Compose([T.Resize(size=(224, 224)),T.ToPILImage()]), 
                                        data_size=9000
                                        )

val_set = WatermarkDataset(val_dir, val_remove,
                                        transform_c=T.Compose([T.Resize(size=(224, 224)), T.ToPILImage()]), 
                                        data_size=1000
                                        )
# test_set = WatermarkDataset(test_dir, test_remove,
#                                         transform_c=T.Compose([T.Resize(size=(224, 224)), T.ToPILImage()]), 
#                                         data_size=1000
#                                         )

In [13]:
len(train_remove)

41

In [14]:
BATCH_SIZE = 8
train_dataloader_img = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader_img = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
# test_dataloader_img = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [15]:
train_file_path = './train.txt'
train_dataset  = TextDataset(train_file_path)
train_iter_w = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False, \
                                                drop_last=True, num_workers=1, collate_fn=collate_fn)
val_dataset = TextDataset('./val.txt')
valid_iter_w = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, \
                                                drop_last=True, num_workers=1, collate_fn=collate_fn)

Loading plain data ...
Loading plain data ...


In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [17]:
# summary(Decoder(64, 1000, 0.1, ))

In [18]:
class PatchEmbedding(nn.Module):
  def __init__(self, in_channels: int=3, patch_size: int=16, emb_size: int = 768, img_size: int = 224):
    self.patch_size = patch_size
    super().__init__()
    self.projection = nn.Sequential(
        nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
#         Rearrange('b e (h) (w) -> b (h w) e'),
    )
    self.cls_token = nn.Parameter(torch.randn(1, 1, emb_size))
    self.positions = nn.Parameter(torch.randn((img_size // patch_size) ** 2, emb_size))

  def forward(self, x: Tensor) -> Tensor:
    b, _, _, _ = x.shape
    x = self.projection(x)
    hp, wp = x.shape[2], x.shape[3]
    x = Rearrange('b e (h) (w) -> b (h w) e')(x)
    x += self.positions # this positions increase the size of the original image
#     print(x.shape)
    return x, hp, wp

In [19]:
summary(PatchEmbedding(in_channels=7), [(10, 7, 224, 224)])

/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Output Shape              Param #
PatchEmbedding                           [10, 196, 768]            151,296
├─Sequential: 1-1                        [10, 768, 14, 14]         --
│    └─Conv2d: 2-1                       [10, 768, 14, 14]         1,377,024
Total params: 1,528,320
Trainable params: 1,528,320
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 2.70
Input size (MB): 14.05
Forward/backward pass size (MB): 12.04
Params size (MB): 5.51
Estimated Total Size (MB): 31.60

In [20]:
print(3*224*224)
print(196*768)

150528
150528


In [21]:
class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size: int = 768, num_heads: int = 8, dropout: float = 0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        # fuse the queries, keys and values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
        # split keys, queries and values in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        # sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** (1/2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

# patches_embedded = PatchEmbedding()(x)
# MultiHeadAttention()(patches_embedded).shape

In [22]:
class ResidualAdd(nn.Module):
  def __init__(self, fn):
    super().__init__()
    self.fn = fn

  def forward(self, x, **kwargs):
    res = x
    x = self.fn(x, **kwargs)
    x += res
    return x

In [23]:
class FeedForwardBlock(nn.Sequential):
  def __init__(self, emb_size: int, expansion: int = 4, drop_p: float = 0.):
    super().__init__(
        nn.Linear(emb_size, expansion * emb_size),
        nn.GELU(),
        nn.Dropout(drop_p),
        nn.Linear(expansion * emb_size, emb_size),
    )

In [24]:
class ClassificationHead(nn.Sequential):
  def __init__(self, emb_size: int=768, n_classes: int = 1000):
    super().__init__(
        Reduce('b n e -> b e', reduction='mean'),
        nn.LayerNorm(emb_size),
        nn.Linear(emb_size, n_classes)
    )

In [25]:
class TransformerEncoderBlock(nn.Sequential):
  def __init__(self,
               emb_size: int = 768,
               drop_p: float = 0.,
               forward_expansion: int = 4,
               forward_drop_p: float = 0.,
               **kwargs):
    super().__init__(
        ResidualAdd(nn.Sequential(
            nn.LayerNorm(emb_size),
            MultiHeadAttention(emb_size, **kwargs),
            nn.Dropout(drop_p)
        )),
        ResidualAdd(nn.Sequential(
            nn.LayerNorm(emb_size),
            FeedForwardBlock(
                emb_size, expansion=forward_expansion, drop_p=forward_drop_p
            ),
            nn.Dropout(drop_p)
        ))
    )

In [26]:
class TransformerEncoder(nn.Sequential):
  def __init__(self, depth: int=12, **kwargs):
    super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])

In [27]:
class ViT(nn.Module):
    def __init__(self,
               in_channels: int = 3,
               patch_size: int = 16,
               emb_size: int = 768,
               img_size: int = 224,
               depth: int = 3,
               **kwargs):
        super().__init__()
        self.patch_embedding = PatchEmbedding(in_channels, patch_size, emb_size, img_size)
        self.transformer_encoder = TransformerEncoder(depth, emb_size=emb_size, **kwargs)

    def forward(self, x:Tensor) -> Tensor:
        b, c, h, w = x.shape[0], x.shape[1], x.shape[2], x.shape[3] 
        x, hp, wp = self.patch_embedding(x)
        x = self.transformer_encoder(x)
        return x

In [28]:
# summary(ViT(), [(2, 3, 224, 224)], device='cpu')
summary(ViT(in_channels=7), [(10, 7, 224, 224)])

Layer (type:depth-idx)                             Output Shape              Param #
ViT                                                [10, 196, 768]            --
├─PatchEmbedding: 1-1                              [10, 196, 768]            151,296
│    └─Sequential: 2-1                             [10, 768, 14, 14]         --
│    │    └─Conv2d: 3-1                            [10, 768, 14, 14]         1,377,024
├─TransformerEncoder: 1-2                          [10, 196, 768]            --
│    └─TransformerEncoderBlock: 2-2                [10, 196, 768]            --
│    │    └─ResidualAdd: 3-2                       [10, 196, 768]            2,363,904
│    │    └─ResidualAdd: 3-3                       [10, 196, 768]            4,723,968
│    └─TransformerEncoderBlock: 2-3                [10, 196, 768]            --
│    │    └─ResidualAdd: 3-4                       [10, 196, 768]            2,363,904
│    │    └─ResidualAdd: 3-5                       [10, 196, 768]            4,723

In [29]:
print(197*768)
print(3*224*224)
'''Do we need the class token? How important is it in our task?'''
'''Class token is used to accumulate global context which helps in classification task.'''

'''How can we validate the ViT representations? Shall we use some pre-trained weights?'''

'''Pass an image and we need image as output itself, how does the scaling using convolution work?
    Do we need to increase and then decrease the feature space?
'''

'''The other thing on the embedder can be to use the ViT itself to learn the features of watermark
instead of Convolutional layers'''

151296
150528


'The other thing on the embedder can be to use the ViT itself to learn the features of watermark\ninstead of Convolutional layers'

In [30]:
class Embedder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.vit_pre = ViT()
        self.vit_embedder = ViT(in_channels=7)
    
    def forward(self, cover_img, watermark):
        # cover image
        batch, c, h, w = cover_img.shape[0], cover_img.shape[1], cover_img.shape[2], cover_img.shape[3]
        
        img_features = self.vit_pre(cover_img)
        img_features = torch.reshape(img_features, (batch, c, h, w))
        
        # text
#         print(watermark.shape)
        w_m = watermark.permute(1, 0, 2)
#         print(w_m.shape)
        w_m = w_m.repeat(1, 49, 1)
        w_m = torch.reshape(w_m, (batch, -1, 224, 224))
        
        watermark = torch.cat([w_m, img_features, cover_img], dim=1) # 64+1+3
#         pdb.set_trace()
        watermark = self.vit_embedder(watermark)
        watermark = torch.reshape(watermark, (batch, c, h, w))
#         watermark = watermark + img_features
        watermark = 0.8*watermark + cover_img # 0.8
        
        return watermark

summary(Embedder(), [(1, 3, 224, 224), (16, 1, 64)])

Layer (type:depth-idx)                                  Output Shape              Param #
Embedder                                                [1, 3, 224, 224]          --
├─ViT: 1-1                                              [1, 196, 768]             --
│    └─PatchEmbedding: 2-1                              [1, 196, 768]             151,296
│    │    └─Sequential: 3-1                             [1, 768, 14, 14]          590,592
│    └─TransformerEncoder: 2-2                          [1, 196, 768]             --
│    │    └─TransformerEncoderBlock: 3-2                [1, 196, 768]             7,087,872
│    │    └─TransformerEncoderBlock: 3-3                [1, 196, 768]             7,087,872
│    │    └─TransformerEncoderBlock: 3-4                [1, 196, 768]             7,087,872
├─ViT: 1-2                                              [1, 196, 768]             --
│    └─PatchEmbedding: 2-3                              [1, 196, 768]             151,296
│    │    └─Sequential: 

In [31]:
class Extractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = ViT()
        self.outer = ClassificationHead(768, 1024)

    def forward(self, watermark_img): 
        features = self.vit(watermark_img)
        out = self.outer(features)
        out = torch.reshape(out, (out.size()[0], 16, 64)) # 16, batch, 64
#         watermark = torch.reshape(watermark, (watermark.size()[0], 8, 64)) # 16, batch, 64
        
        return out.permute(1, 0, 2)

In [32]:
summary(Extractor(), [(1,3,224, 224)])

Layer (type:depth-idx)                                  Output Shape              Param #
Extractor                                               [16, 1, 64]               --
├─ViT: 1-1                                              [1, 196, 768]             --
│    └─PatchEmbedding: 2-1                              [1, 196, 768]             151,296
│    │    └─Sequential: 3-1                             [1, 768, 14, 14]          590,592
│    └─TransformerEncoder: 2-2                          [1, 196, 768]             --
│    │    └─TransformerEncoderBlock: 3-2                [1, 196, 768]             7,087,872
│    │    └─TransformerEncoderBlock: 3-3                [1, 196, 768]             7,087,872
│    │    └─TransformerEncoderBlock: 3-4                [1, 196, 768]             7,087,872
├─ClassificationHead: 1-2                               [1, 1024]                 --
│    └─Reduce: 2-3                                      [1, 768]                  --
│    └─LayerNorm: 2-4        

In [33]:
class ConvBNRelu(nn.Module):
    """
    Building block used in HiDDeN network. Is a sequence of Convolution, Batch Normalization, and ReLU activation
    """
    def __init__(self, channels_in, channels_out, stride=1):

        super(ConvBNRelu, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Conv2d(channels_in, channels_out, 3, stride, padding=1),
            nn.BatchNorm2d(channels_out),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.layers(x)


class Discriminator(nn.Module):
    """
    Discriminator network. Receives an image and has to figure out whether it has a watermark inserted into it, or not.
    """
    def __init__(self):
        super(Discriminator, self).__init__()
        # 3 blocks, 64 channels
        layers = [ConvBNRelu(3, 64)]
        for _ in range(3):
            layers.append(ConvBNRelu(64, 64))

        layers.append(nn.AdaptiveAvgPool2d(output_size=(1, 1)))
        self.before_linear = nn.Sequential(*layers)
        self.linear = nn.Linear(64, 1)

    def forward(self, image):
        X = self.before_linear(image)
        # the output is of shape b x c x 1 x 1, and we want to squeeze out the last two dummy dimensions and make
        # the tensor of shape b x c. If we just call squeeze_() it will also squeeze the batch dimension when b=1.
        X.squeeze_(3).squeeze_(2)
        X = self.linear(X)
        # X = torch.sigmoid(X)
        return X

In [34]:
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
# pretrained_vit = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [35]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 64
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
dropout=0.1

transformer = Transformer(d_model=EMB_SIZE,
           nhead=NHEAD,
           num_encoder_layers=NUM_ENCODER_LAYERS,
           num_decoder_layers=NUM_DECODER_LAYERS,
           dim_feedforward=512,
           dropout=dropout)
positional_encoding = PositionalEncoding(EMB_SIZE, dropout=dropout)

enc = Encoder(EMB_SIZE, TGT_VOCAB_SIZE, transformer, positional_encoding).to(device)
dec = Decoder(EMB_SIZE, TGT_VOCAB_SIZE, transformer, positional_encoding).to(device)

discriminator = Discriminator().to(device)
embedder = Embedder().to(device)
# embedder = Embedder_feat().to(device)
extractor = Extractor().to(device)
# extractor = ExtractorWithNoMixer().to(device)

In [36]:
# summary(dec, [(16, 8, 64), (1, 8), (1, 1)])

In [37]:
# enc.load_state_dict(torch.load('./pretraining/encoder_6_depth_bleu_89.pth'))
# dec.load_state_dict(torch.load('./pretraining/decoder_6_depth_bleu_89.pth'))
enc.load_state_dict(torch.load('./encoder_64_50_noise.pth'))
dec.load_state_dict(torch.load('./decoder_64_50_noise.pth'))

<All keys matched successfully>

In [ ]:
# chkt_pth = './logs_end_to_end/vit_17.pth'
# checkpoint = torch.load(chkt_pth)
# embedder.load_state_dict(checkpoint['embedder_state_dict'])
# extractor.load_state_dict(checkpoint['extractor_state_dict'])
# discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
# # enc.load_state_dict(torch.load('./pretraining/encoder_6_depth_bleu_89.pth'))
# # dec.load_state_dict(torch.load('./pretraining/decoder_6_depth_bleu_89.pth'))

In [ ]:
# handle a way to load pre-trained ViT

Best config is: {'lr': 1e-06, 'enc_lr': 1e-05, 
                 'dec_lr': 0.0001, 'wt_decay': 0.001, 'img_wt': 0.01,
                 'emb_wt': 1.0, 'text_wt': 0.01, 'adv_wt': 0.0001}
                 
- {'lr': 0.0001, 'enc_lr': 1e-07, 'dec_lr': 1e-06, 'wt_decay': 1e-07, 'img_wt': 1.5, 'emb_wt': 1e-05, 'text_wt': 1e-05, 'adv_wt': 0.0001}

- Best config is: {'lr': 1e-06, 'enc_lr': 1e-09, 
                 'dec_lr': 1e-07, 'wt_decay': 1e-07, 'img_wt': 0.5, 
                 'emb_wt': 1e-05, 'text_wt': 1e-05, 'adv_wt': 1e-05}

In [38]:
learning_rate = 1e-4#1e-8#1e-7#1e-4
enc_lr = 1e-6#1e-7# 1e-8
dec_lr = 1e-5#1e-6
wt_decay = 1e-6
params = [
#     {'params':enc.parameters(),'lr':enc_lr},
#     {'params':decoder.generator.parameters()},
#     {'params':decoder.tgt_tok_emb.parameters()},
    {'params':embedder.parameters()},
    {'params':extractor.parameters()}
]
params_discrim = [{'params':discriminator.parameters()}]
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=1)
mse_loss = torch.nn.MSELoss()
bce_with_logits_loss = torch.nn.BCEWithLogitsLoss().to(device)
ssim = SSIM().to(device)
optimizer = optim.Adam(params, lr=learning_rate, weight_decay= wt_decay)
optimizer_discrim = optim.Adam(params_discrim, lr=1e-8)
lr_scheduling = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [39]:
start_symbol=BOS_IDX

In [40]:
def get_rot_mat(theta):
    theta = torch.tensor(theta)
    return torch.tensor([[torch.cos(theta), -torch.sin(theta), 0],
                         [torch.sin(theta), torch.cos(theta), 0]])


def rotate_image_(x, angle):
    dtype =  torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
    
    rot_mat = get_rot_mat(angle)[None, ...].type(dtype).repeat(x.shape[0],1,1)
    grid = F.affine_grid(rot_mat, x.size()).type(dtype)
    x = F.grid_sample(x, grid)
    return x

def gaussian_blur(img, sigma_value):
    b = T.GaussianBlur(kernel_size=(5, 9), sigma=sigma_value)
    return b(img)

def salt_paper(img, amount):
    salt_img = torch.tensor(random_noise(img.detach().cpu().numpy(), mode='salt', amount=amount)).to(device)
    return salt_img

In [41]:
start_symbol=BOS_IDX
def train(dataloader, teacher_forcing_ratio=0.5):
    '''teacher forcing plus greedy decoding method'''
    embedder.train()
    extractor.train()
    enc.eval()
    dec.eval()
#     enc.train()
#     dec.train()
    discriminator.train()
    cover_label = 1
    encoded_label = 0
    train_loss = []
    
    embedding_loss = []
    image_loss = []
    text_loss = []
    ssim_total = []
    adversarial_bce = []
    discr_cover_bce = []
    discr_encod_bce = []
    trg_vocab_size = dec.vocab_size
    
    for data in dataloader:
        img = data[0].to(device)
        sen = data[1].to(device)
#         sen = torch.transpose(sen, -1, 0)
        
        batch_size = sen.shape[1]
        max_len = sen.shape[0]
        trg_vocab_size = dec.vocab_size
        tgt_input = sen[:-1, :]
        
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(sen, sen)
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(device)
    
        with torch.enable_grad():
            optimizer_discrim.zero_grad()
            # train discriminator
            d_target_label_cover = torch.full((batch_size, 1), cover_label, device=device, dtype=torch.float32)
            d_target_label_encoded = torch.full((batch_size, 1), encoded_label, device=device, dtype=torch.float32)
            g_target_label_encoded = torch.full((batch_size, 1), cover_label, device=device, dtype=torch.float32)
            
            d_on_cover = discriminator(img)
            d_loss_on_cover = bce_with_logits_loss(d_on_cover, d_target_label_cover)
            d_loss_on_cover.backward()
            
#             with torch.no_grad():
            encoder_outputs = enc(sen, src_mask, src_padding_mask)
#                 inputs = feature_extractor(img, return_tensors="pt")
#                 outputs_ = pretrained_vit(**inputs)

#             last_hidden_states = outputs_.last_hidden_state
#             img_features = last_hidden_states[:, :-1, :].to(device)
            watermark_img = embedder(img, encoder_outputs)
            # train on fake
            d_on_encoded = discriminator(watermark_img.detach())
            d_loss_on_encoded = bce_with_logits_loss(d_on_encoded, d_target_label_encoded)

            d_loss_on_encoded.backward()
            optimizer_discrim.step()
            
            ssim_img = ssim(watermark_img, img)
    
            d_on_encoded_for_enc = discriminator(watermark_img)
            g_loss_adv = bce_with_logits_loss(d_on_encoded_for_enc, g_target_label_encoded)
            watermark_text = extractor(watermark_img)
            

            ys = torch.ones(1, batch_size).fill_(start_symbol).type(torch.long).to(device)
#             pdb.set_trace()
            for i in range(max_len-1):
                memory = watermark_text.to(device)
                tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                            .type(torch.bool)).to(device)
#                 with torch.no_grad():
                out = dec.decode(ys, memory, tgt_mask)
                out = out.transpose(0, 1)
                prob = dec.generator(out[:, -1])
                outputs[i] = prob
                _, next_word = torch.max(prob, dim=1)
                
                teacher_force = random.random() < 0.5
                if teacher_force:
                    next_word = sen[i]
                ys = torch.cat([ys, next_word.unsqueeze(dim=0)], dim=0)
               
            loss_img = mse_loss(img, watermark_img)

            loss_emb = mse_loss(encoder_outputs, watermark_text)

            text_out = outputs[1:].view(-1, outputs.shape[-1])
    #         text = sen.type(torch.long)
            sen = sen[1:].view(-1)
    #         print(text.device, sen.device)
    #         pdb.set_trace()
            loss_text = loss_fn(text_out, sen)

#             loss = 0.1*loss_img + 0.8*loss_emb + 0.01*loss_text + 0.0001*g_loss_adv
#             loss = 0.01*loss_img + 1.0*loss_emb + 0.01*loss_text + 0.0001*g_loss_adv
            loss = 3*loss_img + 0.5*loss_emb + 0.8*loss_text + 0.0001*g_loss_adv - 0.01*ssim_img
#             loss = 0.5*loss_img + 1e-05*loss_emb + 1e-05*loss_text + 1e-05*g_loss_adv
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#             lr_scheduling.step()
        
        train_loss.append(loss.item())
        embedding_loss.append(loss_emb.item())
        image_loss.append(loss_img.item())
        text_loss.append(loss_text.item())
        ssim_total.append(ssim_img.item())
        adversarial_bce.append(g_loss_adv.item())
        discr_cover_bce.append(d_loss_on_cover.item())
        discr_encod_bce.append(d_loss_on_encoded.item())
  
    return np.mean(train_loss),np.mean(embedding_loss), np.mean(image_loss), np.mean(text_loss), np.mean(ssim_total), np.mean(adversarial_bce), np.mean(discr_cover_bce), np.mean(discr_encod_bce)


def valid(dataloader, teacher_forcing_ratio=0.0):
    embedder.eval()
    extractor.eval()
    enc.eval()
    dec.eval()
    discriminator.eval()
    
    train_loss = []
    cover_label = 1
    encoded_label = 0
    
    embedding_loss = []
    image_loss = []
    text_loss = []
    ssim_total = []
    adversarial_bce = []
    discr_cover_bce = []
    discr_encod_bce = []
    trg_vocab_size = dec.vocab_size
    
    with torch.no_grad():
        for data in dataloader:
            img = data[0].to(device)
            sen = data[1].to(device)
            batch_size = sen.shape[1]
            max_len = sen.shape[0]
            tgt_input = sen[:-1, :]
        
            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(sen, sen)
            d_target_label_cover = torch.full((batch_size, 1), cover_label, device=device, dtype=torch.float32)
            d_target_label_encoded = torch.full((batch_size, 1), encoded_label, device=device, dtype=torch.float32)
            g_target_label_encoded = torch.full((batch_size, 1), cover_label, device=device, dtype=torch.float32)

            outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(device)
            
            # train on real image
            d_on_cover = discriminator(img)
            d_loss_on_cover = bce_with_logits_loss(d_on_cover, d_target_label_cover)
            
            encoder_outputs = enc(sen, src_mask, src_padding_mask)
#             inputs = feature_extractor(img, return_tensors="pt")
#             outputs_ = pretrained_vit(**inputs)

#             last_hidden_states = outputs_.last_hidden_state
#             img_features = last_hidden_states[:, :-1, :].to(device)
            watermark_img = embedder(img, encoder_outputs)
            # train on fake
            d_on_encoded = discriminator(watermark_img)
            d_loss_on_encoded = bce_with_logits_loss(d_on_encoded, d_target_label_encoded)
            
            ssim_img = ssim(watermark_img, img)
            # 1. rotation
#             rotation_angle = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
#             watermark_img = rotate_image_(watermark_img, (np.pi/180)*rotation_angle)
            
            # 2. gaussian blurr
#             watermark_img = gaussian_blur(watermark_img)

            # 3. random additive gaussian noise
#             mean = random.randint(0,2)
#             std = random.randint(0,2)
#             watermark_img = watermark_img + normal_noise(watermark_img.shape, mean, std).to(device)
            
            # on embedder-extractor
            d_on_encoded_for_enc = discriminator(watermark_img)
            g_loss_adv = bce_with_logits_loss(d_on_encoded_for_enc, g_target_label_encoded)
            watermark_text = extractor(watermark_img)
            
            # embedding noise
#             watermark_text = watermark_text + normal_noise(watermark_text.shape).to(device)
            
            ys = torch.ones(1, batch_size).fill_(start_symbol).type(torch.long).to(device)
#             pdb.set_trace()
            for i in range(max_len-1):
                memory = watermark_text.to(device)
                tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                            .type(torch.bool)).to(device)

                out = dec.decode(ys, memory, tgt_mask)
                out = out.transpose(0, 1)
                prob = dec.generator(out[:, -1])
                outputs[i] = prob
                _, next_word = torch.max(prob, dim=1)
                
                teacher_force = random.random() < 0.5
                if teacher_force:
                    next_word = sen[i]
                ys = torch.cat([ys, next_word.unsqueeze(dim=0)], dim=0)
                
            loss_img = mse_loss(img, watermark_img)
            loss_emb = mse_loss(encoder_outputs, watermark_text)

            text_out = outputs[1:].view(-1, outputs.shape[-1])
#             text = sen.type(torch.long)
            sen = sen[1:].view(-1)

            loss_text = loss_fn(text_out, sen)

#             loss = 0.1*loss_img + 0.8*loss_emb + 0.1*loss_text
#             loss = 0.01*loss_img + 1.0*loss_emb + 0.01*loss_text + 0.0001*g_loss_adv
#             loss = 0.1*loss_img + 0.8*loss_emb + 0.01*loss_text + 0.0001*g_loss_adv
#             loss = 0.1*loss_img + 0.1*loss_emb + 1*loss_text #+ 0.001*g_loss_adv
#             loss = 0.5*loss_img + 1e-05*loss_emb + 1*loss_text + 0.0001*g_loss_adv
            loss = 3*loss_img + 0.5*loss_emb + 0.8*loss_text + 0.0001*g_loss_adv - 0.01*ssim_img

            train_loss.append(loss.item())
            embedding_loss.append(loss_emb.item())
            image_loss.append(loss_img.item())
            text_loss.append(loss_text.item())
            ssim_total.append(ssim_img.item())
#             adversarial_bce.append(g_loss_adv.item())
#             discr_cover_bce.append(d_loss_on_cover.item())
#             discr_encod_bce.append(d_loss_on_encoded.item())
  
    return np.mean(train_loss),np.mean(embedding_loss), np.mean(image_loss), np.mean(text_loss), np.mean(ssim_total), np.mean(adversarial_bce), np.mean(discr_cover_bce), np.mean(discr_encod_bce)

In [42]:
def save_checkpoint(model_state, filename):
    torch.save(model_state, filename) 

chkt_pth = './logs_end_to_end/vit_19_50_noise.pth'

In [ ]:
history = {'train_loss': [], 'val_loss': [], 'train_emb_loss': [], 'train_img_loss':[], 'train_text_loss':[],
           'val_emb_loss': [], 'val_img_loss':[], 'val_text_loss':[], 'train_ssim':[], 'val_ssim':[]}

num_epochs=250
# encoder=None
# decoder=None
for epoch in range(num_epochs):
    print("Epoch: ", str(epoch)+'/'+str(num_epochs))
    train_loss, emb_loss, img_loss, text_loss, train_ssim, _, _, _ = train(zip(train_dataloader_img, train_iter_w))
    val_loss, emb_loss_val, img_loss_val, text_loss_val, val_ssim, _, _, _ = valid(zip(val_dataloader_img, valid_iter_w))
    
    model_state = {
        'epoch': epoch,
        'encoder_state_dict': enc.state_dict(),
        'decoder_state_dict': dec.state_dict(),
        'embedder_state_dict': embedder.state_dict(),
        'extractor_state_dict': extractor.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'dis_optim_state_dict': optimizer_discrim.state_dict(),
        'train_loss': train_loss,
        'emb_loss': emb_loss,
        'img_loss': img_loss,
        'ssim': train_ssim,
        'text_loss': text_loss
    }
#     save_checkpoint(model_state, chkt_pth)
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)

    print("Train loss: ", train_loss)
    print("Val loss: ", val_loss)
    history['train_emb_loss'].append(emb_loss)
    history['train_img_loss'].append(img_loss)
    history['train_text_loss'].append(text_loss)
    history['train_ssim'].append(train_ssim)
    
    history['val_emb_loss'].append(emb_loss_val)
    history['val_img_loss'].append(img_loss_val)
    history['val_text_loss'].append(text_loss_val)
    history['val_ssim'].append(val_ssim)

    print("Embedding loss: ", emb_loss)
    print("Image loss: ", img_loss)
    print("Text loss: ", text_loss)
    print("Val Text loss: ", text_loss_val)
    print("Ssim: ", train_ssim)

Epoch:  0/250


/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/data/anaconda3/envs/bishwa_l3/lib/python3

Train loss:  5.841745628145006
Val loss:  5.222255863189697
Embedding loss:  0.764225413163503
Image loss:  0.03860407503694296
Text loss:  6.690085576799181
Val Text loss:  6.021884838104248
Ssim:  0.8319818954070409
Epoch:  1/250


/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data/anaconda3/envs/bishwa_l3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Train loss:  4.262830758836534
Val loss:  3.7788007640838623
Embedding loss:  0.6072110689481099
Image loss:  0.039611137222912575
Text loss:  4.810531454934014
Val Text loss:  4.200661664962769
Ssim:  0.810527108722263
Epoch:  2/250
Train loss:  2.746048644595676
Val loss:  2.4328186559677123
Embedding loss:  0.5093785333633423
Image loss:  0.061222659789853626
Text loss:  2.8941591180165607
Val Text loss:  2.4967127532958986
Ssim:  0.7707700503137377
Epoch:  3/250
Train loss:  1.77737084335751
Val loss:  1.824173373222351
Embedding loss:  0.4911063054667579
Image loss:  0.058435042354795665
Text loss:  1.7054780943658616
Val Text loss:  1.7795986118316651
Ssim:  0.7941702903641594
Epoch:  4/250
Train loss:  1.4314528321160211
Val loss:  1.704262755393982
Embedding loss:  0.4751782041920556
Image loss:  0.049302348696523245
Text loss:  1.3176827246877882
Val Text loss:  1.6603459177017212
Ssim:  0.8261287874115838
Epoch:  5/250
Train loss:  1.291677886221144
Val loss:  1.5599921073913

In [ ]:
epochs = range(1, len(history['train_loss'])+1)
plt.plot(epochs, history['train_emb_loss'], label='Train Embedding loss')
plt.plot(epochs, history['val_emb_loss'], label='Val Embedding loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
epochs = range(1, len(history['train_loss'])+1)
plt.plot(epochs, history['train_img_loss'], label='Train Image loss')
plt.plot(epochs, history['val_img_loss'], label='Val Image loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
epochs = range(1, len(history['train_loss'])+1)
plt.plot(epochs, history['train_text_loss'], label='Train Text loss')
plt.plot(epochs, history['val_text_loss'], label='Val Text loss')
plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
epochs = range(1, len(history['train_loss'])+1)
plt.plot(epochs, history['train_ssim'], label='Train SSIM')
plt.plot(epochs, history['val_ssim'], label='Val SSIM')
plt.title('SSIM')
plt.legend()
plt.show()

- pre-train ViT for image classification task and use few layers in here, this might need more data to train
- or use pre-trained ViT's weight? -> this might decrease our current number of trainable parameters
   - but again for this we might need to use others implementation
- merge Transformer based text encoder-decoder

- train this current model for larger epochs and see the results

## Decoding process in Transformer?

- Autoregressive or not?
- Current issue: I passed all previous outputs by concatenating but decoder produces output for all the given input, meaning it again produces output for the previous output


- https://stackoverflow.com/questions/70146811/feed-decoder-input-in-transformers?rq=3
- How does decoder run multiple times ? confusion or How does the iteration happen?

## Inference

In [ ]:
chkt_pth = './logs_end_to_end/vit_16.pth'
checkpoint = torch.load(chkt_pth)
checkpoint.keys()

In [ ]:
print(checkpoint['ssim'], checkpoint['epoch'], checkpoint['text_loss'])

In [ ]:
torch.save(embedder, 'embedder.pth',)
torch.save(extractor, 'extractor.pth')
torch.save(discriminator, 'discriminator.pth')

In [ ]:
embedder.load_state_dict(checkpoint['embedder_state_dict'])
extractor.load_state_dict(checkpoint['extractor_state_dict'])
enc.load_state_dict(checkpoint['encoder_state_dict'])
dec.load_state_dict(checkpoint['decoder_state_dict'])

In [ ]:
def predict(dataset, samples=1):
    embedder.eval()
    extractor.eval()
    enc.eval()
    dec.eval()

    trg_vocab_size = dec.vocab_size
    data = next(iter(dataset))
    print(data[0].shape, data[1].shape)
    img = data[0].to(device)
    sen = data[1].to(device)
#     sen = sen.permute(1, 0)
    
    batch_size = sen.shape[1]
    max_len = sen.shape[0]
    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(sen, sen)

#     outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(device)
    outputs = torch.zeros(max_len, batch_size).to(device)
    
    encoder_outputs = enc(sen, src_mask, src_padding_mask)
    watermark_img_no_noise = embedder(img, encoder_outputs)
    watermark_img = watermark_img_no_noise

    # 1. rotation
#     watermark_img = rotate_image_(watermark_img, (np.pi/180)*5)

    # 2. gaussian blurr
#     watermark_img = gaussian_blur(watermark_img, 1)

    # 3. crop
#     watermark_img = center_crop(watermark_img, 20)

#     5. salt paper
#     watermark_img = salt_paper(watermark_img, 0.1)

    # 6. cropout
#         watermark_img = cropout(watermark_img, [0.1, 0.4], [0.1, 0.4])

    watermark_text = extractor(watermark_img)
    loss_emb = mse_loss(encoder_outputs, watermark_text)
    # embedding noise
#         watermark_text = watermark_text + normal_noise(watermark_text.shape).to(device)
#         watermark_text = watermark_text + random_noise(watermark_text, -0.5, 0.5)

    ys = torch.ones(1, batch_size).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = watermark_text.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(device)
#             pdb.set_trace()
        out = dec.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = dec.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        outputs[i] = next_word

        ys = torch.cat([ys, next_word.unsqueeze(dim=0)], dim=0)

    ssim_img = ssim(watermark_img_no_noise, img).detach().cpu().numpy()
#     ssim_img = 0
    watermark = watermark_img.detach().cpu().numpy()
    img = img.detach().cpu().numpy()
    watermark = np.moveaxis(watermark, 1, -1)
    img = np.moveaxis(img, 1, -1)

    watermark_img_no_noise = watermark_img_no_noise.detach().cpu().numpy()
    watermark_img_no_noise = np.moveaxis(watermark_img_no_noise, 1, -1)
    
#     pdb.set_trace()
    outputs = outputs.type(torch.int)
    outputs = torch.transpose(outputs, -1, 0)
    outputs = outputs.detach().cpu().numpy().tolist()
    
    inputs = torch.transpose(sen, -1, 0)
    inputs = inputs.detach().cpu().numpy().tolist()
    #     batch_size = sentence.shape[1]
#     pdb.set_trace()
    translated_sentence = []
    input_sentence = []
    for i in range(batch_size):
        token_ = outputs[i]
        in_ = inputs[i]
        sen = vocab_transform[TGT_LANGUAGE].lookup_tokens(token_)
        in_sen = vocab_transform[TGT_LANGUAGE].lookup_tokens(in_)
        translated_sentence.append(' '.join(x for x in sen[1:]))
        input_sentence.append(' '.join(x for x in in_sen[1:]))

    return img, watermark, watermark_img_no_noise, input_sentence, translated_sentence, np.mean(ssim_img), encoder_outputs, watermark_text

In [ ]:
# img, watermark, watermark_img_no_noise, input_sentence, output_sentence, ssim_ = predict(zip(train_dataloader_img, train_iter_w))
img, watermark, watermark_img_no_noise, input_sentence, output_sentence, ssim_, e_o, w_t = predict(zip(val_dataloader_img, valid_iter_w))
# img, watermark, watermark_img_no_noise, input_sentence, output_sentence, ssim = predict(zip(test_dataloader_img, test_iter_w))
for i in range(len(input_sentence)):
    print("Input: ", input_sentence[i].split('<eos>')[0].split('<pad>')[0])
#     print(input_sentence[i])
    print("Output: ", output_sentence[i].split('<eos>')[0])
    print("**"*20)

In [ ]:
ssim_

In [ ]:
for i in range(len(input_sentence)):
        fig = plt.figure(figsize=(12, 6))

        fig.add_subplot(241)
        plt.title('cover image')
        plt.axis('off')
        l = len(input_sentence[i].split('<eos>')[0].split('<pad>')[0])
        print("Input sentence: ", input_sentence[i].split('<eos>')[0].split('<pad>')[0])
        plt.imshow(img[i])
        
#         fig.add_subplot(242)
#         plt.title('No noise watermarked image')
#         plt.axis('off')
#         plt.imshow(watermark_img_no_noise[i])
        
        fig.add_subplot(242)
        plt.title('watermark image')
        plt.axis('off')
        print('**'*10)
        print("Output sentence: ", output_sentence[i].split('<eos>')[0][:l])
        plt.imshow(watermark[i])

        plt.show()

In [ ]:
def calculate_blue_score(input_sentence, predicted_sentence):
    input_token = []
    predicted_token = []
    unk = ['<unk>']
    for in_, pre_ in zip(input_sentence, predicted_sentence):
        input_sen = in_.split('<eos>')[0].split('<pad>')[0]
#         l = len(input_sen)
#         input_sen = in_.strip().split('<eos>')[0].split('<pad>')[0]
        input_sen = input_sen.split(' ')
        
        n = [i for i in range(len(input_sen)) if input_sen[i] in unk]
        input_sen = ' '.join(word for word in input_sen if word not in unk).strip()
        l = len(input_sen.split(' '))
        
        predicted_sen = pre_.split('<eos>')[0].split(' ')[:l]
#         predicted_sen = predicted_sen.strip().split(' ')
        c = 0
        for i in n:
            try:
                del predicted_sen[i-c]
                c += 1
            except:
                del predicted_sen[i-1]
            
        input_token.append([input_sen.split(' ')])
        predicted_token.append(predicted_sen)
        
    return bleu_score(predicted_token, input_token)

In [ ]:
calculate_blue_score(input_sentence, output_sentence)

# Analyzing encoder outputs and extractor output

In [ ]:
x = torch.tensor([[[2, 3]]], dtype=torch.float32)
y = torch.tensor([[[4, 5]]], dtype=torch.float32)
torch.square(y - x)

In [ ]:
torch.mean(torch.square(y-x), dim=2)

In [ ]:
print(e_o.shape, w_t.shape)

In [ ]:
torch.mean(torch.mean(torch.square(e_o - w_t), dim=2), dim=1)

## TODO:
- Analyze recent architecture
- Hyperparameter tuning

In [ ]:
# https://ai.googleblog.com/2020/04/optimizing-multiple-loss-functions-with.html